### Armar un sistema de recomendación de películas. 

Éste consiste en recomendar películas a los usuarios basándose en películas similares, por lo que se debe encontrar la similitud de puntuación entre esa película y el resto de películas. 

Se ordenarán según el score de similaridad y devolverá:
- una lista de Python con 5 valores, 
- cada uno siendo el string del nombre de las películas con mayor puntaje 
- en orden descendente.

In [1]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors

In [10]:
df = pd.read_csv('C:/Users/TheFercho/Desktop/SoyHenry/Labs/PI_TheFercho_labs/ML_dataset.csv')

In [11]:
df.columns

Index(['id', 'title', 'genres', 'original_language', 'spoken_languages',
       'popularity', 'production_companies', 'production_countries',
       'release_year', 'vote_count', 'vote_average'],
      dtype='object')

In [21]:
# Análisis de Componentes Principales (PCA). El PCA es una técnica de reducción de dimensionalidad que 
# transforma un conjunto de variables correlacionadas en un conjunto más pequeño de variables no correlacionadas 
# llamadas componentes principales.

def recomendacion_4(titulo):
    # Cargar el dataframe
        
    # Obtener el índice de la película que coincide con el título ingresado (case insensitive)
    indices = pd.Series(df.index, index=df['title'].str.lower()).drop_duplicates()
    titulo_lower = titulo.lower()
    
    if titulo_lower not in indices:
        return "Título no encontrado"
    
    idx = indices[titulo_lower]
    
    # Seleccionar las características relevantes para el algoritmo KNN
    features = ['popularity', 'vote_average', 'vote_count', 'genres']
    X = df[features]
    
    # Convertir las listas en columnas binarias utilizando MultiLabelBinarizer
    mlb = MultiLabelBinarizer()
    encoded_genres = pd.DataFrame(mlb.fit_transform(X['genres']), columns=mlb.classes_, index=X.index)
    
    # Convertir los nombres de características numéricas a cadenas
    numeric_features = X.drop('genres', axis=1)
    numeric_features.columns = numeric_features.columns.astype(str)
    
    # Aplicar PCA para reducir la dimensionalidad
    pca = PCA(n_components=0.9)  # Mantenemos el 90% de la varianza explicada
    encoded_genres_pca = pca.fit_transform(encoded_genres)
    
    # Concatenar las características numéricas y las características transformadas por PCA
    X = pd.concat([numeric_features, pd.DataFrame(encoded_genres_pca)], axis=1)
    
    # Normalizar las características
    X = (X - X.mean()) / X.std()
    
    # Crear una instancia del algoritmo KNN
    knn = NearestNeighbors(n_neighbors=6)  # Consideramos 6 vecinos, incluyendo la película seleccionada
    
    # Resolvemos el siguiente error con la recomendacion de la propia libreria:  
    # Feature names are only supported if all input features have string names, but your input has ['int', 'str'] 
    # as feature name / column name types. If you want feature names to be stored and validated, 
    # you must convert them all to strings, by using: =>>>>> X.columns = X.columns.astype(str)
    X.columns = X.columns.astype(str)
    
    # Entrenar el modelo KNN
    knn.fit(X)
    
    # Obtener la distancia y los índices de los vecinos más cercanos
    distances, indices = knn.kneighbors(X.iloc[idx].values.reshape(1, -1))
    
    # Obtener los títulos de las películas más similares (excluyendo la película seleccionada)
    similar_movies = df['title'].iloc[indices[0][1:6]]
    
    return similar_movies


In [26]:
recomendacion_4('toy story')

c:\Users\TheFercho\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


21128    Despicable Me 2
15335        Toy Story 3
4751      Monsters, Inc.
11559        Ratatouille
359        The Lion King
Name: title, dtype: object